# Equivariant Machine Learned Interatomic Potentials Models

Created by ***Debsundar Dey*** and ***Tejus Rohatgi***

This tutorial aims to demonstrate two state-of-the-art MLIP models, **NequIP** and **MACE**.

## Table of Contents
0. [Before Starting](#prep)
1. [Dataset](#dataset)
    *   [BOTNET Dataset](#botnet)
    *   [Visualising Dataset](#visualise)
2. [NequIP](#nequip)
    * [Introduction](#nequip-intro)
    * [Installing NequIP](#nequip-install)
    * [Configuration](#nequip-config)
    * [Training NequIP](#nequip-training)
    * [Molecular Dynamics using NequIP](#nequip-md)
3. [MACE](#mace)
    * [Introduction](#mace-intro)
    * [Installing MACE](#mace-install)
    * [Configuration](#mace-config)
    * [Training MACE](#mace-training)
    * [Molecular Dynamics using MACE](#mace-md)
4. [Exercises](#exercises)
5. [BONUS: Visualizing `.xyz` files](#ovito)
    





## 0. Before Starting <a name="prep"></a>

In [ ]:
# Install necessary packages and dependencies
!pip install ase
!pip install e3nn==0.4.4
!pip install opt_einsum
!pip install torch_ema
!pip install prettytable

In [ ]:
# Import necessary libraries
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import ase
import random
import time
import pylab as pl
import torch

random.seed(123)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device.type

## 1. Dataset <a name="dataset"></a>
The dataset plays a crucial role in training and evaluating Machine Learning Interatomic Potential (MLIP) models. Its quality, diversity, and relevance significantly impact the performance and generalizability of the MLIP. The dataset provides the atomic configurations and corresponding properties (e.g., energy, forces, stress) that the MLIP aims to predict.



### 1.1. BOTNET Dataset <a name="botnet"></a>
We will be working on the [BOTNET](https://github.com/davkovacs/BOTNet-datasets) collection of dataset used in the paper by [Batatia et.al](https://arxiv.org/abs/2205.06643). The collection contains the following datasets:

1. **Ethanol and Methanol:** Consists of two training datasets, first taken revMD17 dataset and was sampled from a long 500 K Ab Initio molecular dynamics trajectory and the second training set contains the 1000 ethanol geometries of the first training set but is augmented by 300 K methanol geometries also sampled from 500 K Ab Initio molecular dynamics simulation.
2. **3BPA:** Dataset contains snapshots of a large flexible drug-like organic molecule sampled from different temperature molecular dynamics trajectories.
3. **Acetylacetone:** Dataset contains data related to acetylacetone (acac), specifically focusing on proton transfer reactions between its tautomeric forms. This dataset includes configurations obtained from Ab Initio molecular dynamics simulations at 300 K and 600 K.

In [ ]:
# Downloading BOTNet Datset
!git clone https://github.com/davkovacs/BOTNet-datasets.git

In [ ]:
!ls BOTNet-datasets/dataset_ethanol

In [ ]:
!ls BOTNet-datasets/dataset_3BPA

In [ ]:
!ls BOTNet-datasets/dataset_acac

We will be working with **[Acetylacetone dataset](https://github.com/davkovacs/BOTNet-datasets/tree/main/dataset_acac)** in this tutorial.



### 1.2. Visualising Dataset <a name="visualise"></a>

We will utilise the `ase` package to get more information about the datset we are looking on. Let's start with `dataset_acac/train_300K.xyz`.



In [ ]:
from ase.io import read, write

# Load the dataset
dataset = read("BOTNet-datasets/dataset_acac/train_300K.xyz", index=":")
dataset[0]

In [ ]:
num_configs = len(dataset)
print(f"Number of configurations in the dataset: {num_configs}")

Let's utilise `ase.visualize` module to visualize the first few configurations in the datset.

In [ ]:
from ase.visualize.plot import plot_atoms
import matplotlib.pyplot as plt

# Create nine subplots with an increased figure size
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(10, 6), sharex=True, sharey=True)

for i, ax_i in enumerate(ax.flat):  # using ax.flat to iterate through the subplots
    plot_atoms(dataset[i], ax_i)
    ax_i.axis('off')
    ax_i.set_xlim(0, 10)  # To zoom
    ax_i.set_ylim(0, 10)  # To zoom
plt.show()

## 2. NequIP <a name="nequip"></a>




### 2.1. Introduction <a name="nequip-intro"></a>

Welcome to the *NequIP Tutorial with Molecular Dynamics*, where we’ll train a NequIP model with AIMD data and use it for MD simulations! Think of this as a speedrun through the molecular jungle—we might skip some scenic routes (a.k.a. deep physical intuition), but we’ll get you to the finish line with a working model. So buckle up, and let’s dive into the NequIPverse.

#### About NequIP

NequIP employs E(3)-equivariant graph neural networks, which respect the symmetries of three-dimensional Euclidean space (rotations, translations, and reflections). This allows the model to learn both scalar (e.g., energy) and vector (e.g., forces) properties in a symmetry-aware manner.

### 2.2. Installing NequIP <a name="nequip-install"></a>

In [ ]:
# Clone NequIP
!git clone https://github.com/mir-group/nequip.git

# Install NequIP
!cd nequip && pip install .

In [ ]:
!ls nequip

### 2.3. Configuration <a name="nequip-config"></a>

In [ ]:
# Specify Paths
nequip_dir = Path(os.getcwd()) / 'nequip'
nequip_configs_dir = nequip_dir / 'configs'
nequip_results_dir = nequip_dir / 'results'

# Create directories if not exists
os.makedirs(nequip_configs_dir, exist_ok=True)
os.makedirs(nequip_results_dir, exist_ok=True)

#### 2.3.1. Hyperparameters in Training NequIP <a name="nequip-hypparam"></a>

In [ ]:
# Specifying parameters for training a NequIP model

parameters_nequip = {

    # Specify model name and save directories
    "root": str(nequip_results_dir),
    "run_name": "botnet",

    # Dataset
    "dataset": "ase",
    "dataset_file_name": "BOTNet-datasets/dataset_acac/train_300K.xyz",

    "key_mapping": {
        "z": "atomic_numbers",
        "E": "total_energy",
        "F": "forces",
        "R": "pos",
    },
    "chemical_symbols": ["H", "O", "C"],

    # Hyperparameters
    "num_features": 64, # Number of channels determining the size of the model
    "r_max": 4.0,  # Maximum interaction radius
    "l_max": 1, # Symmetry of messages
    "parity": "true", # whether to include features with odd mirror parity
    "num_layers": 4,  # Number of message-passing layers in the model
    # radial network
    "num_basis": 8, # number of basis functions used in the radial basis, 8 usually works best
    "invariant_layers": 2, # number of radial layers, usually 1-3 works best, smaller is faster
    "invariant_neurons": 64,  # number of hidden neurons in radial function, smaller is faster

    # Optimization
    "n_train": "95%", # Fraction of the training set to use as training
    "n_val": "5%", # Fraction of the training set to use as validation
    "batch_size": 10, # Batch size for training
    "max_epochs": 20, # Maximum number of epochs
    "default_dtype": "float64", # Precision for calculations
    "model_dtype": "float32",
    "wandb": False, # Logging
    "device": device.type, # Device to use ("cuda" for GPU)
    "seed": 123,
    "dataset_seed": 456,
}


# Save parameters to a YAML file
input_nequip = nequip_configs_dir / "input.yaml"

with open(input_nequip, "w") as file:
    yaml.dump(parameters_nequip, file, default_flow_style=False)

print(f"Parameters have been saved to {input_nequip}")


### 2.4. Training NequIP <a name="nequip-train"></a>

To train a NequIP model, you can use the `nequip-train` file.

In [ ]:
!nequip-train --help

In [ ]:
!nequip-train nequip/configs/input.yaml

**Extra:** Try running the model with different hyperparameters.

In [ ]:
# Check the output files
nequip_train_dir = nequip_results_dir / parameters_nequip["run_name"]
for file in nequip_train_dir.rglob("*"):
    print(file)

Check the file saved as `metric_epoch.csv` for information of about training.

In [ ]:
nequip_train_csv = nequip_train_dir / "metrics_epoch.csv"

# Load the CSV file
df = pd.read_csv(nequip_train_csv)

df.head()

In [ ]:
# Columns to plot
metrics = [
    # 'training_loss_e',
    'training_loss',
    # 'training_e_mae',
    # 'training_e_rmse',
    # 'validation_loss_e',
    'validation_loss',
    # 'validation_e_mae',
    # 'validation_e_rmse',
    ]

# Plot each metric against the epoch
plt.figure(figsize=(10, 6))
for metric in df.columns:
    if metric in metrics:
        plt.plot(df['epoch'], df[metric], label=metric)

# Add labels, title, and legend
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('NequIP Training')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

### 2.5. Molecular Dynamics using NequIP <a name="nequip-md"></a>


Molecular dynamics (MD) is a computer simulation technique used to model and analyze the physical movements of atoms and molecules over time. By applying the principles of classical mechanics, specifically Newton's equations of motion, MD calculates the trajectories of particles based on the forces acting between them. These forces are typically derived from potential energy functions that describe interactions such as bonding, electrostatics, and van der Waals forces. The MLIPs are the potential energy functions we are using here.

#### 2.5.1. ASE Molecular Dynamics <a name="nequip-md-ase"></a>

We will be using ASE (Atomic Simulation Environment) for performing a simple MD simulations using the module [`ase.md`](https://wiki.fysik.dtu.dk/ase/ase/md.html).

The function `simpleMD()` is based on this [tutorial](https://colab.research.google.com/drive/1ZrTuTvavXiCxTFyjBV4GqlARxgFwYAtX).

In [ ]:
from ase import units
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import Stationary, ZeroRotation, MaxwellBoltzmannDistribution
from ase.calculators.calculator import Calculator
from IPython import display

def simpleMD(
    init_conf: ase.Atoms,
    temp: float,
    calc: Calculator,
    fname: str,
    s: int,
    T: int,
    seed = 123,
    init_temp = 300,
    friction = 0.1,
    ) -> None:
    """
    Perform a simple molecular dynamics (MD) simulation using the Langevin thermostat.

    Parameters:
    -----------
    init_conf : Initial atomic configuration to start the simulation.
    temp :  Target temperature of the system in Kelvin.
    calc :  Calculator to compute forces and energies for the simulation.
    fname :  Filename to store the trajectory (.xyz format).
    s :  Interval (in steps) to save frames and update the plot.
    T :  Total number of MD steps to run.
    seed: To ensure reproducibiity
    init_temp: Initialization temperature
    friction: Damping factor in Langevin

    Returns: None
    """

    # Assign the calculator for force and energy calculations
    init_conf.set_calculator(calc)

    # Initialize the temperature of the system
    random.seed(123)  # Seed for reproducibility
    MaxwellBoltzmannDistribution(init_conf, temperature_K=init_temp)  # Initial temp 300 K
    Stationary(init_conf)  # Remove net momentum
    ZeroRotation(init_conf)  # Remove angular momentum

    # Set up the Langevin thermostat
    dyn = Langevin(init_conf, 1.0 * units.fs, temperature_K=temp, friction=friction)

    # Required for inline plotting in Jupyter Notebook
    %matplotlib inline

    # Initialize lists to store time, temperature, and energy
    time_fs = []
    temperature = []
    energies = []

    # Remove any existing trajectory file with the same name
    os.system("rm -rfv " + fname)

    # Create a plot with two subplots
    fig, ax = pl.subplots(2, 1, figsize=(10, 6), sharex="all", gridspec_kw={"hspace": 0, "wspace": 0})

    def write_frame():
        """
        Function to save a frame of the trajectory and update the live plot.
        """
        dyn.atoms.write(fname, append=True)  # Append current configuration to file
        time_fs.append(dyn.get_time() / units.fs)  # Store current simulation time
        temperature.append(dyn.atoms.get_temperature())  # Store current temperature
        energies.append(dyn.atoms.get_potential_energy() / len(dyn.atoms))  # Potential energy per atom

        # Update the plot
        ax[0].plot(np.array(time_fs), np.array(energies), color="b")
        ax[0].set_ylabel("E (eV/atom)")

        ax[1].plot(np.array(time_fs), temperature, color="r")
        ax[1].set_ylabel("T (K)")
        ax[1].set_xlabel("Time (fs)")
        ax[1].axhline(y=temp, linestyle = '--', c = "black")

        # Clear the previous plot and display the updated one
        display.clear_output(wait=True)
        display.display(pl.gcf())
        time.sleep(0.01)  # Pause for a short time to make the updates visible

    # Attach the write_frame function to be called every `s` steps
    dyn.attach(write_frame, interval=s)

    # Run the MD simulation and time its execution
    t0 = time.time()
    dyn.run(T)
    t1 = time.time()

    print("MD finished in {0:.2f} minutes!".format((t1 - t0) / 60))

We use Langevin thermostat using [`ase.md.langevin.Langevin`](https://wiki.fysik.dtu.dk/ase/ase/md.html#module-ase.md.langevin).

[Langevin dynamics](https://en.wikipedia.org/wiki/Langevin_dynamics) is a computational simulation method used to model the behavior of particles in a system by incorporating both deterministic and stochastic (random) forces. It extends classical molecular dynamics (MD) by introducing additional terms to account for the interaction of particles with their surrounding environment, typically represented as a heat bath. This approach is particularly useful for simulating systems at a constant temperature and for capturing the effects of thermal fluctuations and friction without explicitly modeling every component of the environment.

\begin{equation}
M \ddot{\mathbf{X}} = -\nabla U(\mathbf{X}) - \gamma M \dot{\mathbf{X}} + \sqrt{2 M \gamma k_{\mathrm{B}} T} \, \mathbf{R}(t),
\end{equation}


#### 2.5.2. Deploying NequIP Model <a name="nequip-md-deploy"></a>
The `nequip-deploy` command is used to deploy the result of a training session into a model that can be stored and used for inference. It compiles a NequIP model trained in Python to TorchScript. The result is an optimized model file that has no dependency on the nequip Python library, or even on Python itself

In [ ]:
!nequip-deploy --help

In [ ]:
!nequip-deploy build --train-dir nequip/results/botnet/ nequip/results/botnet/deployed_model.pth

#### 2.5.3. NequIP Calculator <a name="nequip-md-calc"></a>
NequIP models can run molecular dynamics or geometry optimisation through the ASE calculator.

In [ ]:
init_conf = read('./BOTNet-datasets/dataset_acac/test_MD_300K.xyz', index=0)
init_conf

In [ ]:
from nequip.ase import NequIPCalculator

# Specify the path to the pre-trained MACE model
nequip_model = nequip_train_dir / "deployed_model.pth"

# Initialize the NequIP calculator with the specified model
nequip_calc = NequIPCalculator.from_deployed_model(nequip_model)

# Call the simpleMD function to perform a molecular dynamics simulation
simpleMD(init_conf, temp=300, calc=nequip_calc, fname='nequip/results/nequip_md_acac_300K.xyz', s=10, T=500)

## 3. MACE <a name="mace"></a>

### 3.1. Introduction <a name="mace-intro"></a>
#### About MACE

[MACE](https://proceedings.neurips.cc/paper_files/paper/2022/hash/4a36c3c51af11ed9f34615b81edb5bbc-Abstract-Conference.html) is a machine learning interatomic potential (MLIP) framework that leverages **message-passing neural networks (MPNNs)** with equivariant representations to capture **complex many-body interactions** in atomic systems.

Inspired from [ACE](https://doi.org/10.1103/PhysRevB.99.014104)-framework, MACE implements a many-body order basis and combines it with message passing framework similar to NequIP to resulting in a fast and highly parallelizable model, reaching or exceeding state-of-the-art accuracy, reducing the number of message passing iterations to just two. For a deeper dive refer to [this paper](https://arxiv.org/abs/2205.06643).

#### Objective
This tutorial aims to give a brief introduction to using MACE for training on dataset and implementing a simple Molecular Dynamics code.

### 3.2. Installing MACE <a name="mace-install"></a>

In [ ]:
# Clone MACE
!git clone --depth 1 https://github.com/ACEsuit/mace.git

# Install MACE
!cd mace && pip install .

In [ ]:
# View MACE directory
!ls mace

### 3.3. Configuration <a name="mace-config"></a>

In [ ]:
# Specify paths
mace_dir = Path(os.getcwd()) / 'mace'
mace_configs_dir = mace_dir / 'configs'
mace_results_dir = mace_dir / 'results'

# Create directories if not exists
os.makedirs(mace_configs_dir, exist_ok=True)
os.makedirs(mace_results_dir, exist_ok=True)

#### 3.3.1 Hyperparameters in Training MACE

For configuring the hyperparameters, we can specify a YAML file. The following is an example:

In [ ]:
# Specifying parameters for training a MACE model

parameters_mace = {

    # Specify model name and save directories
    "name": "botnet",  # Name of the model being trained
    "model_dir": str(mace_results_dir),
    "log_dir": str(mace_results_dir),
    "checkpoints_dir": str(mace_results_dir),
    "results_dir": str(mace_results_dir),

    # Train file specifications
    "train_file": "BOTNet-datasets/dataset_acac/train_300K.xyz",  # Path to training data
    "forces_key": "forces",  # Key for forces in the dataset
    "energy_key": "energy",  # Key for energy in the dataset

    # From isolated_atoms.xyz get the reference energies for isolated atoms.
    "E0s": {
        1: -13.663181292231226,  # Energy for Hydrogen (H)
        6: -1029.2809654211628,  # Energy for Carbon (C)
        8: -2042.0330099956639,  # Energy for Oxygen (O)
    },

    # Hyperparameters
    "model": "MACE",  # Type of model to train (https://mace-docs.readthedocs.io/en/latest/guide/training.html#model)
    "num_channels": 64, # Number of channels determining the size of the model
    "r_max": 4.0,  # Maximum interaction radius
    "max_L": 1, # Symmetry of messages

    "num_interactions": 2,  # Number of message-passing layers in the model (recommended to keep default)
    "correlation": 3, # The order of the many-body expansion (recommended to keep default)
    "max_ell": 3, # The angular resolution describes how well the model can describe angles (recommended to keep default)

    # Optimization
    "valid_fraction": 0.05, # Fraction of the training set to use as validation
    "batch_size": 10, # Batch size for training
    "max_num_epochs": 20, # Maximum number of epochs
    "default_dtype": "float32", # Precision for calculations
    "device": "cuda", # Device to use ("cuda" for GPU)
    "seed": 123,
}

# Save parameters to a YAML file
input_mace = mace_configs_dir / "input.yaml"

with open(input_mace, "w") as file:
    yaml.dump(parameters_mace, file, default_flow_style=False)

print(f"Parameters have been saved to {input_mace}")

### 3.4. Training MACE <a name="mace-train"></a>

To train a MACE model, you can use the `run_train.py` script.

In [ ]:
!python3 ./mace/scripts/run_train.py --config="mace/configs/input.yaml"

**Extra:** Try running the model with different hyperparameters.

In [ ]:
# Check the output files
mace_train_dir = Path(parameters_mace["model_dir"])
for file in mace_train_dir.rglob("*"):
    print(file)

Check the file saved as `..._train.txt` for information of about training.

In [ ]:
mace_train_txt = mace_train_dir / f"{parameters_mace['name']}_run-{parameters_mace['seed']}_train.txt"

# Parsing the file
eval_data = {}
opt_data = {}

with open(mace_train_txt, 'r') as file:
    lines = file.readlines()

    for line in lines:
        try:
            record = json.loads(line)

            if record["mode"] == "eval":
                # Extract the epoch
                epoch = record.get("epoch", None)
                epoch_key = epoch if epoch is not None else None

                # Initialize a sub-dictionary for this epoch if not already present
                if epoch_key not in eval_data:
                    eval_data[epoch_key] = {}

                # Add all other keys (except 'mode' and 'epoch') to the sub-dictionary
                for key, value in record.items():
                    if key not in ["mode", "epoch"]:
                        eval_data[epoch_key][key] = value

            elif record["mode"] == "opt":
                # Extract the epoch
                epoch = record.get("epoch", None)
                epoch_key = epoch if epoch is not None else None

                # Initialize a sub-dictionary for this epoch if not already present
                if epoch_key not in opt_data:
                    opt_data[epoch_key] = {}

                # Initialize lists for all keys including 'epoch' if not present
                for key in record:
                    if key not in ["mode", "epoch"]: # Removed "epoch" to initialize it
                        if key not in opt_data[epoch_key]:  # This line is crucial
                            opt_data[epoch_key][key] = []

                # Append values to the lists
                for key, value in record.items():
                    if key not in ["mode", "epoch"]:
                        opt_data[epoch_key][key].append(value)

        except json.JSONDecodeError:
            continue

In [ ]:
opt_data[0]

In [ ]:
eval_data[0]

The `..._train.txt` file contains two types of data:

1. Optimization Data: Batch Optimization data
2. Evaluation Data: Validation data



In [ ]:
train_loss_data = []
valid_loss_data = []

for epoch in opt_data.keys():
  train_loss_data.append(opt_data[epoch]['loss'])
  valid_loss_data.append(eval_data[epoch]['loss'])

average_train_loss_data=[]

num_validation_configs = parameters_mace['valid_fraction'] * num_configs
num_train_configs = num_configs - num_validation_configs

for loss in train_loss_data:
  average_train_loss_data.append(np.sum(loss) / num_train_configs)

plt.plot(valid_loss_data, marker='v', linestyle='-', label='validation_loss')
plt.plot(average_train_loss_data, marker='o', linestyle='-', label='training_loss')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('MACE Training')
plt.legend()
plt.grid(True)
plt.show()

### 3.5. Molecular Dynamics using MACE <a name="mace-md"></a>

#### 3.5.1 MACE Calculator
MACE models can run molecular dynamics or geometry optimisation through the ASE calculator. There is no need for model deployement unlike in NequIP.

In [ ]:
init_conf = read('./BOTNet-datasets/dataset_acac/test_MD_300K.xyz', index=0)
init_conf

In [ ]:
from mace.calculators import MACECalculator

# Specify the path to the pre-trained MACE model
mace_model = mace_train_dir / f"{parameters_mace['name']}_compiled.model"

# Initialize the MACE calculator with the specified model
mace_calc = MACECalculator(model_paths=[mace_model], device='cuda', default_dtype="float32")

# Call the simpleMD function to perform a molecular dynamics simulation
simpleMD(init_conf, temp=300, calc=mace_calc, fname='mace/results/mace_md_acac_300K.xyz', s=10, T=500)

## 4. Exercises <a name="exercises"></a>

1. We haven't tested the trained NequIP or MACE model. To test it we can use:

  - Use `nequip-evaluate` for NequIP model. [Refer this](https://github.com/mir-group/nequip?tab=readme-ov-file#evaluating-trained-models-and-their-error) or use `nequip-evaluate --help` for more reference.
  - Use `eval_configs.py` for MACE model. [Refer this](https://mace-docs.readthedocs.io/en/latest/guide/evaluation.html).

2. Play with the hyperparamters for NequIP and MACE. How are they similar? What are the differences in the models?

3. Training was done on `train_300K.xyz`. If you see the `dataset_acac` directory, there are other training datasets. Train it on `train_600K.xyz` and perform evaluation and MD on `test_600K.xyz`. How could you change the MD simulation for this dataset? Compare the results from tutorials.



## 5. BONUS: Visualizing `.xyz` files <a name="ovito"></a>

We can visualize the Molecular Dynamics file using softwares such as [OVITO](https://www.ovito.org/).

1. This can be done by downloading and installing OVITO software on your local device from the following website - [Download OVITO](https://www.ovito.org/#download)

2. After downloading it, open the application. [Refer this for usage instructions](https://www.ovito.org/manual).

3. Download `.xyz` trajectory files to your local device.

4. Go to `Files`->`Load File...`-> Load the Trajectory File.

5. Play the animation.



